# Your First (Simulated) Robot!
In this notebook you will run Mujoco, a physics simulator that we will be using for the class. You shoul study the API for Mujoco as we will be using the python wrappers which directly map to the C code it was written in. 

Mujoco works in the following way: A model and data structure compose the simulator. The model provides the geometric, kinematic, structural information while any data e.g., generalized coordinates, velocities, contacts, etc., is stored in the data structure. The data is created from the model and a model is created from a configuration file which defines the robot the environment parameters. 

We create a robot model by using an `.xml` file which is provided in the A1 folder. The `.xml` file defines the simulation parameters, the structure of the robot, transforms, joints, dynamics, geometry, sensors, actuators, and anything else in the environment. Take a look at the composition of the one provided by the A1 robot in the a1 directory. These files are similar to the more common `.urdf` files which stands for Universal Robot Description Format. These files, like `.xml` describe the robot and the environment and can be converted from one to another. In many other simulators and robotics toolbox, a `.urdf` is often used to describe the robot. There are a few advantages and disadvantages for using either format. If you download the source code for mujoco from Deepmind, they provide with a `convert` function which can transform a `.urdf` to a `.xml` file and vice-versa. I recommend exploring both files which are provided in this repo. 

This notebook will go through the steps of importing, creating a model, data, and visualizing your first robot. 

## Mujoco imports
Below are the typical imports you will need to get started with. 

In [1]:
import mujoco
import mujoco_viewer
import numpy as np

## Creating model and data structures
A model can be created through many ways. One can even specify a string that has the structure of the xml file that we can use to build a robot. The data is then built from the modal and used to extract out computation from simulating the system forward. 

In [2]:
model = mujoco.MjModel.from_xml_path('./a1/xml/a1.xml')
data = mujoco.MjData(model)

In [ ]:
mujoco.mj_forward(model, data)
q0 = np.copy(data.qpos)
q0vel = np.cops(data.qvel)





# ----- in mppi code 

data.qpos = q0.copy()
data.qvel = q0vel.copy()
mujoco.mj_forward(model, data)

## Visualization
The following code will create a openGL window which will render the environment to you. However, just running it will not do anything. As part of the code, you will have to run `viewer.render()` each time after you step the simulation forward. Depending on how often you run the render function, the visualization will appear faster/slower than real-time. The true time stepping is defined in the `.xml` file. 

In [3]:
# create the viewer object
viewer = mujoco_viewer.MujocoViewer(model, data)

In [4]:
# PD control parameters
kp = 80.
kd = 10.
# simulate and render
i = 0 # <-- counter 
for _ in range(10000): 
    with open("mocap.txt","r") as filestream: # < -- grabs the mocap data
        for line in filestream:
            # the following lines convert the text to target joint values
            currentline = line.split(",")
            frame = currentline[0]
            t = currentline[1]
            target_joints = np.array([float(_q) for _q in currentline[2:14]])
            
            # make sure that you can still see the environment 
            if viewer.is_alive:
                # pull the joint position and velocities
                q = np.array(data.qpos[7:]) # <-- notice that the first 7 values are the base body position (3) and orientation in quaternains (4)
                v = np.array(data.qvel[6:]) # <-- here the joint velocities start after the base body rotations (only 3, not 4)
                # basic PD controller 
                tau = kp * (target_joints-q) + kd * (-v)
                # to apply a control input to mujoco, you need to set the inputs to the data structure's ctrl variable
                data.ctrl = tau
                # the data and model sets passed to the step function to step the simulation. New poses, vel, cont, etc, get recorded into data
                mujoco.mj_step(model, data)
                
                # every 30 steps render the environment 
                if i % 30 == 0:
                    viewer.render()
                i += 1
            else:
                break

# close
viewer.close()

Pressed ESC
Quitting.


# Writing your own robot model 
The mujoco xml format relies on basic transforms (rotations and translations) to build a robot. 

### Setting the objects in the environment
The robot is defined within the *worldbody* xml tag. Here we define the geometry of the objects with the tag *geom* and bodies are defined as *body*. Note that a body needs a geom which you specify within the body tag. There are several kinds of geometries you can define (and even custom ones using an .stl file). 

### Setting joints
Rotations, translations, and free body movements are defined through the *joint* tag. There are several kinds of joints that are common for robotics. Prismatic (linear), revolute (rotation), spherical (roll pitch yaw), and many more. You should explore the mujoco xml documentation and see the kinds that exist. Each joint will have a specific set of flags that you need to fill out and reference frames that connect bodies together. 

### Setting reference frames
It is often beneficial to have a reference frame attached to a body which can be accessed at any time (e.g., think of the location of a camera or sensor). We use the *site* flag within the body tag to specify where a site is located. Note that the position and orientation are specified locally. 

### Actuators 
Mujoco has several actuators (as do other simulation engines). To specify an actuator one must has a joint to place the actuator on. Check out the example below on a double pendulum and check out the xml file to see how to setup actuators. As with joints there are several kinds of actuators. The type will vary from application to application and some are more appropriate than others and depend on how the robot is designed. 

### Explaining the XML code below
The code below is what an xml file to describe your robot looks like. We can also import the code as a string into mujoco so you can create the robot straight from your code and edit how it looks and is configured. Compare this code with the a1.xml file located in this repository. 

In [19]:
robot_model = """
<!-- double pendulum model
    The state space is populated with joints in the order that they are
    defined in this file. The actuators also operate on joints.
    State-Space (name/joint/parameter):
        - pole1      hinge       angle (rad)
        - pole2      hinge       angle (rad)
        - pole1      hinge       angular velocity (rad/s)
        - pole2      hinge       angular velocity (rad/s)

        Actuators (name/actuator/parameter):
        - pole1      motor       force tau1 (N)
        - pole2      motor       force tau2 (N)
-->
<mujoco model="double pendulum">
    <compiler coordinate="local" inertiafromgeom="true"/>
    <custom>
        <numeric data="2" name="frame_skip"/>
    </custom>
    <default>
        <joint damping="0.05"/>
        <geom contype="0" friction="1 0.1 0.1" rgba="0.7 0.7 0 1"/>
    </default>
    <option gravity="0.0 0.0 -9.81" integrator="RK4" timestep="0.01"/>
    <size nstack="3000"/>
    <worldbody>
        <geom name="floor" pos="0 0 -3.0" rgba="0.8 0.9 0.8 1" size="40 40 40" type="plane"/>
        <body name="pole1" pos="0 0 0">
            <joint axis="0 1 0" name="hinge1" pos="0 0 0" type="hinge"/>
            <geom fromto="0 0 0 0 0 0.6" name="cpole" rgba="0 0.7 0.7 1" size="0.045 0.3" type="capsule"/>
            <body name="pole2" pos="0 0 0.6">
                <joint axis="0 1 0" name="hinge2" pos="0 0 0" type="hinge"/>
                <geom fromto="0 0 0 0 0 0.6" name="cpole2" rgba="0 0.7 0.7 1" size="0.045 0.3" type="capsule"/>
                <site name="tip" pos="0 0 .6" size="0.01 0.01"/>
            </body>
        </body>
    </worldbody>
    <actuator>
        <motor ctrllimited="true" ctrlrange="-1 1" gear="500" joint="hinge1" name="hinge1"/>
        <motor ctrllimited="true" ctrlrange="-1 1" gear="500" joint="hinge2" name="hinge2"/>
    </actuator>
</mujoco>
"""

In [20]:
model = mujoco.MjModel.from_xml_string(robot_model)
data = mujoco.MjData(model)

In [21]:
viewer = mujoco_viewer.MujocoViewer(model, data)

In [22]:
# simulate and render
while True: 
            
    # make sure that you can still see the environment 
    if viewer.is_alive:
        # to apply a random control input to mujoco, you need to set the inputs to the data structure's ctrl variable
        data.ctrl = np.random.normal(0., 0.01, size=(2,))
        # the data and model sets passed to the step function to step the simulation. New poses, vel, cont, etc, get recorded into data
        mujoco.mj_step(model, data)
        viewer.render()
    else:
        break

# close
viewer.close()